# Temporal  Tables
## Creación

In [ ]:
CREATE TABLE dbo.Temporal
(
  TemporalID INT NOT NULL IDENTITY(1,1) PRIMARY KEY
, FirstName VARCHAR(100) NOT NULL
, LastName VARCHAR(100) NOT NULL
, StartTime DATETIME2(7) GENERATED ALWAYS AS ROW START HIDDEN 
, EndTime DATETIME2(7) GENERATED ALWAYS AS ROW END HIDDEN
, PERIOD FOR SYSTEM_TIME ([StartTime], [EndTime])
)
WITH
(
SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.TemporalHistory )
)


## Insertar Datos

In [ ]:
INSERT INTO dbo.Temporal
    (
    FirstName, LastName
    )
SELECT Distinct FirstName, LastName
FROM person.Person 


## Verificar Datos

In [ ]:
SELECT * FROM dbo.Temporal

### Incluir Columnas de Version

In [ ]:
SELECT *, StartTime, EndTime FROM dbo.Temporal

## Modificar Datos

In [ ]:
UPDATE dbo.Temporal
set FirstName = 'Bob', LastName = 'May'
where TemporalID = 33

### Verificar Cambio en tabla corriente

In [ ]:
SELECT * , StartTime, EndTime FROM dbo.Temporal
where TemporalID = 33

### Verificar cambios en tabla historica

In [ ]:
SELECT * FROM dbo.TemporalHistory
where TemporalID = 33

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33

### Cambiemos el nombre una vez mas

In [ ]:
UPDATE dbo.Temporal
set FirstName = 'John', LastName = 'Smith'
where TemporalID = 33

### Visualizemos de otra manera

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

## Borrando Datos

In [ ]:
Delete from dbo.Temporal where TemporalID = 33

## Reinsertando Datos

In [ ]:
SET IDENTITY_INSERT dbo.Temporal ON

INSERT INTO dbo.Temporal
(TemporalID, FirstName ,LastName)
select Top (1) TemporalID, FirstName, LastName from TemporalHistory
ORDER by StartTime desc

SET IDENTITY_INSERT dbo.Temporal OFF

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

## System_Time

### Between

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME BETWEEN '2020-06-10 22:16' and '2020-06-10 22:24'
where TemporalID = 33
order by StartTime desc

### AS OF

In [ ]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME AS OF '2020-06-10 22:16' 
where TemporalID = 33
order by StartTime desc

# Modificando la Tabla Principal

## Agregando una Columna

In [ ]:
ALTER TABLE dbo.Temporal
add StrongID UNIQUEIDENTIFIER DEFAULT(NewID())

In [21]:
select * from dbo.TemporalHistory

(3 rows affected)

Total execution time: 00:00:00.003

TemporalID,FirstName,LastName,StartTime,EndTime,StrongID
33,Evan,Adams,2020-06-10 22:14:47.0568050,2020-06-10 22:16:38.4111668,NULL
33,Bob,May,2020-06-10 22:16:38.4111668,2020-06-10 22:20:58.6225728,NULL
33,John,Smith,2020-06-10 22:20:58.6225728,2020-06-10 22:24:55.2785476,NULL


In [22]:
Update dbo.Temporal set StrongID = NEWID()
where StrongID is null

(19515 rows affected)

Total execution time: 00:00:00.252

In [25]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime, StrongID,
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

(5 rows affected)

Total execution time: 00:00:00.008

TemporalID,FirstName,LastName,StartTime,EndTime,StrongID,RowType
33,John,Smith,2020-06-10 22:38:23.8354045,9999-12-31 23:59:59.9999999,95f56153-3b07-444d-8b12-4e9ca2506fd2,CURRENT
33,John,Smith,2020-06-10 22:26:35.6995792,2020-06-10 22:38:23.8354045,NULL,OUTDATED
33,John,Smith,2020-06-10 22:20:58.6225728,2020-06-10 22:24:55.2785476,NULL,OUTDATED
33,Bob,May,2020-06-10 22:16:38.4111668,2020-06-10 22:20:58.6225728,NULL,OUTDATED
33,Evan,Adams,2020-06-10 22:14:47.0568050,2020-06-10 22:16:38.4111668,NULL,OUTDATED


## Agregando una columna NOT-NULL

### Veamos un intento fallido 

In [26]:
ALTER TABLE dbo.Temporal
ADD [TimeStamp] RowVersion NOT NULL

: Msg 515, Level 16, State 2, Line 1
Cannot insert the value NULL into column 'TimeStamp', table 'AdventureWorks2017.dbo.TemporalHistory'; column does not allow nulls. UPDATE fails.

The statement has been terminated.

Total execution time: 00:00:00.007

In [27]:
ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = OFF)

ALTER TABLE dbo.TemporalHistory
ADD [TimeStamp] RowVersion NOT NULL

ALTER TABLE dbo.Temporal
ADD [TimeStamp] RowVersion NOT NULL

ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.TemporalHistory, DATA_CONSISTENCY_CHECK = ON ))

Commands completed successfully.

Total execution time: 00:00:00.286

In [28]:
SELECT TemporalID, FirstName, LastName, StartTime, EndTime, StrongID, [TimeStamp],
Case EndTime
when '9999-12-31 23:59:59.9999999' then 'CURRENT'
ELSE 'OUTDATED' END AS RowType
FROM dbo.Temporal
FOR SYSTEM_TIME ALL
where TemporalID = 33
order by StartTime desc

(5 rows affected)

Total execution time: 00:00:00.016

TemporalID,FirstName,LastName,StartTime,EndTime,StrongID,TimeStamp,RowType
33,John,Smith,2020-06-10 22:38:23.8354045,9999-12-31 23:59:59.9999999,95f56153-3b07-444d-8b12-4e9ca2506fd2,0x0000000000010CCD,CURRENT
33,John,Smith,2020-06-10 22:26:35.6995792,2020-06-10 22:38:23.8354045,NULL,0x0000000000010CAC,OUTDATED
33,John,Smith,2020-06-10 22:20:58.6225728,2020-06-10 22:24:55.2785476,NULL,0x000000000000C025,OUTDATED
33,Bob,May,2020-06-10 22:16:38.4111668,2020-06-10 22:20:58.6225728,NULL,0x000000000000C024,OUTDATED
33,Evan,Adams,2020-06-10 22:14:47.0568050,2020-06-10 22:16:38.4111668,NULL,0x000000000000C023,OUTDATED


# LIMPIANDO EL HISTORIAL

In [ ]:
ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = OFF)

TRUNCATE TABLE dbo.TemporalHistory

ALTER TABLE dbo.Temporal
SET (SYSTEM_VERSIONING = ON ( HISTORY_TABLE = dbo.TemporalHistory, DATA_CONSISTENCY_CHECK = ON ))

# DATA_CONSISTENCY_CHECK = ON 

Verifica que la tabla de historial propuesta cumpla con lo siguiente:
- No Identidades
- No Indices independientes
- No Constraints
- Tabla Principal tenga una llave primaria pero la historial no. 
- CDC no este encendido en la tabla historica
- No llaves foraneas


# Convertir Tabla Existente en Temporal

In [31]:
EXECUTE dbo.TableToTemporal @Table2PartName = 'person.Person'

person.Person Is not a candidate for Temporal Table Conversion

Total execution time: 00:00:00.002

In [32]:
Alter Table person.Person 
set (system_versioning = off)



Commands completed successfully.

Total execution time: 00:00:00.007

In [33]:
alter table person.person 
drop period for SYSTEM_TIME

Commands completed successfully.

Total execution time: 00:00:00.006

In [34]:
drop table person.person_history

Commands completed successfully.

Total execution time: 00:00:00.007

# RESET - DO NOT RUN

In [ ]:
alter TABLE dbo.Temporal
set (system_versioning = off)
DROP TABLE dbo.Temporal
DROP TABLE dbo.TemporalHistory